In [22]:
import requests
import os
import shutil
import numpy as np
import math
import glob

def fetch_pdb(pdb_id):
    """
    Fetches a PDB file from the RCSB database.
    Args:
        pdb_id (str): The PDB ID of the structure.
    Returns:
        str: The raw PDB file data.
    """
    url = f"https://files.rcsb.org/download/{pdb_id.upper()}.pdb"
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        raise Exception(f"Failed to fetch PDB file for {pdb_id}. HTTP Status: {response.status_code}")

def pdb_to_xyz(pdb_data, pdb_id):
    """
    Converts PDB data to XYZ format, adding the PDB ID as the second line.
    Args:
        pdb_data (str): The raw PDB file data.
        pdb_id (str): The PDB ID to be added as the second line in the XYZ file.
    Returns:
        str: XYZ formatted string.
    """
    xyz_lines = []
    with open (pdb_data, "r") as file:
        for line in file:
            if line.startswith("ATOM") or line.startswith("HETATM"):
                atom_name = line[12:16].strip()
                element = line[76:78].strip()
                x = line[30:38].strip()  # Original precision X
                y = line[38:46].strip()  # Original precision Y
                z = line[46:54].strip()  # Original precision Z

                # Clean up atomic information and use full precision from the PDB file
                xyz_line = f"{element} {x} {y} {z}"
                xyz_lines.append(xyz_line)

        # Write the number of atoms followed by the PDB ID and atom lines
        xyz_output = f"{len(xyz_lines)}\n"
        xyz_output += f"{pdb_id.upper()}\n"  # PDB ID as the second line
        xyz_output += "\n".join(xyz_lines)
    return xyz_output

def save_xyz(output_dir, pdb_id, xyz_data):
    """
    Saves the XYZ data to a file in the given output directory.
    Args:
        output_dir (str): The directory where the output XYZ file should be saved.
        pdb_id (str): The PDB ID to be used in the file name.
        xyz_data (str): The XYZ formatted string.
    """
    os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist
    output_filename = os.path.join(output_dir, f"{pdb_id.upper()}.xyz")
    with open(output_filename, 'w') as file:
        file.write(xyz_data)
    print(f"XYZ data saved to {output_filename}")

#def main(pdb_id, output_dir):
#    pdb_data = fetch_pdb(pdb_id)
#    xyz_data = pdb_to_xyz(pdb_data, pdb_id)
#    save_xyz(output_dir, pdb_id, xyz_data)


#pdb_id = "1n11"  # Example: "1TUP"
#output_dir = "/Users/ntw/Desktop/MHT_Calculations/" +pdb_id
#main(pdb_id, output_dir)


In [24]:
def parse_xyz(xyz_filename):
    """
    Parses an XYZ file and returns atom data along with indices of BR and CL atoms.
    Args:
        xyz_filename (str): The name of the XYZ file.
    Returns:
        list: List of all atoms with element names and coordinates.
        list: Indices of BR atoms.
        list: Indices of CL atoms.
    """
    atoms = []
    br_atoms = []
    cl_atoms = []

    with open(xyz_filename, 'r') as file:
        lines = file.readlines()[2:]  # Skip the first two lines (atom count and PDB ID)
        
        for i, line in enumerate(lines):
            parts = line.split()
            element = parts[0]
            x, y, z = parts[1], parts[2], parts[3]

            # Add atom data to list
            atoms.append((element, x, y, z))

            # Keep track of BR and CL atom indices
            if element == "Br":
                br_atoms.append(i)
            elif element == "BR":
                br_atoms.append(i)
            elif element == "Cl":
                cl_atoms.append(i)
            elif element == "CL":
                cl_atoms.append(i)

    return atoms, br_atoms, cl_atoms

def swap_elements(atoms, br_idx, cl_idx):
    """
    Swaps a single BR atom with a single CL atom in the atom list.
    Args:
        atoms (list): List of atoms (element, x, y, z).
        br_idx (int): Index of the BR atom to swap.
        cl_idx (int): Index of the CL atom to swap.
    Returns:
        list: A new list of atoms with the BR and CL swapped.
    """
    swapped_atoms = atoms[:]

    # Swap BR with CL at the specified indices
    br_atom = swapped_atoms[br_idx]
    cl_atom = swapped_atoms[cl_idx]

    # Swap their elements
    swapped_atoms[br_idx] = ("Cl", br_atom[1], br_atom[2], br_atom[3])
    swapped_atoms[cl_idx] = ("Br", cl_atom[1], cl_atom[2], cl_atom[3])

    return swapped_atoms

def write_xyz(filename, atoms):
    """
    Writes the list of atoms to an XYZ file.
    Args:
        filename (str): The name of the output XYZ file.
        atoms (list): List of atoms (element, x, y, z).
    """
    with open(filename, 'w') as file:
        file.write(f"{len(atoms)}\n\n")  # First line: number of atoms
        for atom in atoms:
            file.write(f"{atom[0]} {atom[1]} {atom[2]} {atom[3]}\n")


def generate_swapped_files(xyz_filename, output_dir):
    """
    Generates new XYZ files with each BR-CL swap in a separate file.
    Args:
        xyz_filename (str): The input XYZ filename.
        output_dir (str): The directory where the new XYZ files will be saved.
    """
    # Parse the original XYZ file
    atoms, br_atoms, cl_atoms = parse_xyz(xyz_filename)

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Generate all swaps (one BR with one CL)
    for br_idx in br_atoms:
        for cl_idx in cl_atoms:
            # Swap the BR and CL atoms at these indices
            swapped_atoms = swap_elements(atoms, br_idx, cl_idx)

            # Generate a unique output filename for each swap
            output_filename = os.path.join(output_dir, f"swapped_{br_idx}_{cl_idx}_{os.path.basename(xyz_filename)}")
            write_xyz(output_filename, swapped_atoms)
            print(f"Swapped XYZ data saved to {output_filename}")

#def main():
#    xyz_filename = "/Users/ntw/Desktop/MHT_Calculations/1N11/1n11.xyz"
#    output_dir = "/Users/ntw/Desktop/MHT_Calculations/1N11/"
#    generate_swapped_files(xyz_filename, output_dir)
#
#if __name__ == "__main__":
#    main()


In [26]:
def process_directory(dir_path):
    """
    Method to process a directory.
    Replace this with your desired logic.
    """
    print(f"Processing directory: {dir_path}")

def traverse_directories(root_dir):
    """
    Traverses the directory tree starting from root_dir and 
    applies the process_directory method to each directory.
    """
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for subdirname in dirnames:
            subdir_path = os.path.join(dirpath, subdirname)
            try:
                make_DFTB_input(subdir_path)
            except Exception as e:
                print("Skipping " + subdir_path)
                pass
#traverse_directories("/Users/ntw/Desktop/MHT_Calculations/")


In [28]:
def filter_xyz(xyz_filename):
    """
    Parses an XYZ file and returns atom data (without BR and CL atoms).
    Args:
        xyz_filename (str): The name of the XYZ file.
    Returns:
        list: List of atoms (element, x, y, z) excluding BR and CL.
    """
    atoms = []

    with open(xyz_filename, 'r') as file:
        lines = file.readlines()[2:]  # Skip the first two lines (atom count and PDB ID)

        for line in lines:
            parts = line.split()
            element = parts[0]

            # If the element is BR or CL, skip this atom
            if element not in ["BR", "CL"]:
                x, y, z = parts[1], parts[2], parts[3]
                atoms.append((element, x, y, z))

    return atoms

def write_filtered_xyz(filename, atoms):
    """
    Writes the list of atoms to an XYZ file.
    Args:
        filename (str): The name of the output XYZ file.
        atoms (list): List of atoms (element, x, y, z).
    """
    with open(filename, 'w') as file:
        file.write(f"{len(atoms)}\n\n")  # First line: number of atoms
        for atom in atoms:
            file.write(f"{atom[0]} {atom[1]} {atom[2]} {atom[3]}\n")

def remove_br_cl_atoms(xyz_filename, output_dir):
    """
    Removes all BR and CL atoms from the XYZ file and saves the result to a new file.
    Args:
        xyz_filename (str): The input XYZ filename.
        output_dir (str): The directory where the new XYZ file will be saved.
    """
    # Parse the original XYZ file, excluding BR and CL atoms
    atoms = filter_xyz(xyz_filename)

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Write the filtered atoms to a new XYZ file
    output_filename = os.path.join(output_dir, f"no_BR_CL_{os.path.basename(xyz_filename)}")
    write_filtered_xyz(output_filename, atoms)
    print(f"Filtered XYZ data saved to {output_filename}")

#def main():
#    xyz_filename = "/Users/ntw/Desktop/MHT_Calculations/1N11/1n11.xyz"
#    output_dir = "/Users/ntw/Desktop/MHT_Calculations/1N11/"
#    remove_br_cl_atoms(xyz_filename, output_dir)
#
#if __name__ == "__main__":
#    main()


In [30]:

def move_files_to_subdirectories(directory):
    """Moves each file in a directory to its own subdirectory named after the file.

    Args:
        directory: The path to the directory containing the files.
    """
    for filename in os.listdir(directory):
        source_path = os.path.join(directory, filename)
        if ".xyz" in filename:
            if os.path.isfile(source_path):
                file_base_name, _ = os.path.splitext(filename)
                destination_dir = os.path.join(directory, file_base_name)
                os.makedirs(destination_dir, exist_ok=True)
                destination_path = os.path.join(destination_dir, filename)
                shutil.move(source_path, destination_path)
        else: 
            pass

# Example usage:
#directory_path = "/path/to/your/directory"  # Replace with the actual path to your directory
#move_files_to_subdirectories(directory_path)

In [32]:

def make_ChimeraX_Scripts(parent_directory):
    # Define the parent directory you want to search through

    # Create the run_all.py script in the parent directory
    run_all_script_path = parent_directory + 'run_all.py'

    with open(run_all_script_path, 'w') as f:
        # Write the import statement at the beginning of the script
        f.write("from chimerax.core.commands import run\n\n")
        
        # Iterate through all subdirectories and files in the parent directory
        for filename in glob.glob(os.path.join(parent_directory, "*.pdb")):
            full_pdb_path = os.path.join(parent_directory, filename)
            print(full_pdb_path)
                
                # Define the new filename for the PDB file with "_added_h" added before the .pdb extension
            new_pdb_path = os.path.splitext(full_pdb_path)[0] + '_added_h.pdb'
                
                # Add the commands to the run_all.py script
            f.write(f"open {full_pdb_path}\n")
            f.write(f"addh\n")
            f.write(f"save \"{new_pdb_path}\"\n")
            f.write(f"close all\n")
        
    print(f"Created {run_all_script_path} to process all PDB files!")
    return None

In [34]:


# Function to calculate the Euclidean distance between two points in 3D space
def calculate_distance(atom1, atom2):
    return math.sqrt((atom1[0] - atom2[0]) ** 2 + (atom1[1] - atom2[1]) ** 2 + (atom1[2] - atom2[2]) ** 2)

# Function to read .xyz file and check for atoms closer than 0.3 Ångströms
def check_atoms_closer_than_threshold(file_path, threshold=0.3):
    with open(file_path, 'r') as file:
        # Read the number of atoms and skip the comment line
        num_atoms = int(file.readline().strip())
        file.readline()
        
        atoms = []
        
        # Read the atoms and their coordinates
        for line in file:
            parts = line.split()
            atom_type = parts[0]
            x, y, z = float(parts[1]), float(parts[2]), float(parts[3])
            atoms.append((atom_type, x, y, z))
        
        # Check distances between all pairs of atoms
        for i in range(len(atoms)):
            for j in range(i + 1, len(atoms)):
                distance = calculate_distance(atoms[i][1:], atoms[j][1:])
                if distance < threshold:
                    print(f"Atoms {atoms[i][0]} at {atoms[i][1:]} and {atoms[j][0]} at {atoms[j][1:]} are {distance:.4f} Ångströms apart")

# Usage example
#check_atoms_closer_than_threshold('/Users/ntw/Desktop/MHT_Calculations/1K63/1K63/1K63.xyz')


# Function to read an XYZ file
def read_xyz(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    num_atoms = int(lines[0].strip())
    atoms = []
    for line in lines[2:]:
        parts = line.split()
        atoms.append([parts[0], float(parts[1]), float(parts[2]), float(parts[3])])
    return atoms

# Function to write an XYZ file


# Function to calculate the distance between two atoms
def calculate_Euclid_distance(atom1, atom2):
    return np.sqrt((atom1[1] - atom2[1])**2 + (atom1[2] - atom2[2])**2 + (atom1[3] - atom2[3])**2)

# Function to process the atoms and remove those that are too close
def remove_close_atoms(atoms, threshold=0.2):
    indices_to_remove = set()
    
    # Check each pair of atoms
    for i in range(len(atoms)):
        for j in range(i+1, len(atoms)):
            distance = calculate_Euclid_distance(atoms[i], atoms[j])
            if distance < threshold:
                # Mark the atom with the higher index for removal
                indices_to_remove.add(j)
    
    # Remove atoms that are too close
    filtered_atoms = [atom for i, atom in enumerate(atoms) if i not in indices_to_remove]
    
    return filtered_atoms

# Main function
def process_xyz(input_filename, output_filename, threshold=0.3):
    atoms = read_xyz(input_filename)
    filtered_atoms = remove_close_atoms(atoms, threshold)
    write_xyz(output_filename, filtered_atoms)

# Example usage
#process_xyz("/Users/ntw/Desktop/MHT_Calculations/1K63/1K63/1K63.xyz", \
#"/Users/ntw/Desktop/MHT_Calculations/1K63/1K63/1K63_corrected.xyz")
#check_atoms_closer_than_threshold('/Users/ntw/Desktop/MHT_Calculations/1K63/1K63/1K63.xyz')

import os
def traverse_check_distance(root_dir):
    """
    Traverses the directory tree starting from root_dir and 
    applies the process_directory method to each directory.
    """
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for subdirname in dirnames:
            subdir_path = os.path.join(dirpath, subdirname)
            try:
                print(subdir_path+"/"+os.path.basename(subdirname)+"/"+os.path.basename(subdirname)+".xyz")
                check_atoms_closer_than_threshold(subdir_path+"/"+os.path.basename(subdirname)+"/"+os.path.basename(subdirname)+".xyz")
            except Exception as e:
                print("Skipping " + subdir_path)
                pass
#traverse_check_distance("/Users/ntw/Desktop/MHT_Calculations/")
    



In [36]:
def make_DFTB_input(foldername):
        OUTPUT_FILENAME = 'dftb_in.hsd'
    
        folder = foldername
    
        if not folder:
            print('Cancelled - No valid folder entered')
            return
        try:
            geometryinput = [f for f in os.listdir(folder) if f.endswith('.xyz') or f.endswith('.gen')]
        except:
            print('Cancelled - No .xyz or .gen geometry file found.')
            return
    # =============================================================================
    #     Driver = sg.Window('Driver', [[sg.T('Use conjugate gradient driver for geometry optimization?')], [sg.Yes(s=10), sg.No(s=10)]], disable_close=True).read(close=True)
    #     LevelofTheory = sg.Window('Level of Theory', [[sg.T('Use DFTB3 level of theory?')], [sg.Yes(s=10), sg.No(s=10)]], disable_close=True).read(close=True)
    #     SKfolder = sg.popup_get_folder('Folder for Slater-Koster files.',
    #                               title='DFTB+ GUI')
    # =============================================================================
    
        outfile = open(os.path.join(foldername, OUTPUT_FILENAME), 'w')
    
    #Read and write the geometry file to dftb_in.hsd
        containsSulfur = 0
        containsOxygen = 0
        containsCarbon = 0
        containsChlorine = 0
        containsHydrogen = 0
        containsNitrogen = 0
        containsPhosphorus = 0
        containsBromine = 0
        containsMagnesium = 0
        containsZinc = 0
        input_as_list=[]
        for i, file in enumerate(geometryinput):
            with open(os.path.join(folder, file), "r") as input:
                if file.endswith('.gen'):
                    outfile.write('Geometry=GenFormat{')
                if file.endswith('.xyz'):
                    outfile.write('Geometry=xyzFormat{ \n')
                    for line in input:
                        input_as_list.append(line)
                        outfile.write(line)
                    secondlineonwards=input_as_list[2:]
                #for line in secondlineonwards:
                #        outfile.write(line)
                              
        if 'S' in str(secondlineonwards):
            containsSulfur=1
            print('Input file contains Sulfur')
        if 'C' in str(secondlineonwards):
            containsCarbon=1
            print('Input file contains Carbon')
        if 'H' in str(secondlineonwards):
            containsHydrogen=1
            print('Input file contains Hydrogen')
        if 'O' in str(secondlineonwards):
            containsOxygen=1
            print('Input file contains Oxygen')
        if 'N' in str(secondlineonwards):
            containsNitrogen=1
            print('Input file contains Nitrogen')
        if 'Cl' in str(secondlineonwards):
            containsChlorine=1
        if 'CL' in str(secondlineonwards):
            containsChlorine=1
            print('Input file contains Chlorine')
        if 'P' in str(secondlineonwards):
            containsPhosphorus=1
            print('Input file contains Phosphorus')
        if 'Br' in str(secondlineonwards):
            containsBromine=1
            print('Input file contains Bromine')
        if 'BR' in str(secondlineonwards):
            containsBromine=1
            print('Input file contains Bromine')
        if 'Mg' in str(secondlineonwards):
            containsMagnesium=1
            print('Input file contains Magnesium')
        if 'MG' in str(secondlineonwards):
            containsMagnesium=1
            print('Input file contains Magnesium')        
        if 'Zn' in str(secondlineonwards):
            containsZinc=1
            print('Input file contains Zinc') 
        if 'ZN' in str(secondlineonwards):
            containsZinc=1
            print('Input file contains Zinc')    
                            
    #        outfile.write('}')
    #       if Driver == "('Yes', {})": 
    #           outfile.write("Driver = ConjugateGradient {\n", "MovedAtoms = 1:-1\n", "MaxSteps = -1\n", 'OutputPrefix = "geom.out"/n', '}')
    #       if Driver == "('No', {})":
    #           outfile.write("Driver = GradientDescent {\n", "MovedAtoms = 1:-1\n", "MaxSteps = -1\n", 'OutputPrefix = "geom.out"/n', '}')
        outfile.write('}\n')  
        outfile.write(
    'Hamiltonian = DFTB { \n'
    ' Scc = Yes \n'
    ' MaxSCCIterations = 2000 \n'
    ' SlaterKosterFiles = Type2FileNames { \n'
    '  Prefix = "/home/ntw/TDW_123_MHT_Calculations/MHT_Calculations/3ob-main/skfiles/\n'
    '  Separator = "-" \n'
    '  Suffix = ".skf" \n'
    '  LowerCaseTypeName = No \n'
    '  } \n')
        outfile.write('\n')
        outfile.write(' MaxAngularMomentum { \n')
        if containsCarbon == 1:
                outfile.write('  C = "p" \n')
        if containsHydrogen == 1:
                outfile.write('  H = "s" \n')
        if containsOxygen == 1:
                outfile.write('  O = "p" \n')
        if containsNitrogen == 1:
                outfile.write('  N = "p" \n')
        if containsPhosphorus == 1:
                outfile.write('  P = "d"/n')
        if containsSulfur == 1:
                outfile.write('  S = "d" \n')
        if containsChlorine == 1:
                outfile.write('  Cl = "d" \n')
        if containsBromine == 1:
                outfile.write('  Br = "d" \n')
        if containsMagnesium == 1:
                outfile.write('  Mg = "p" \n')
        if containsZinc == 1:
                outfile.write('  Zn = "d" \n')
        outfile.write('  }')
        outfile.write('\n')
        outfile.write(' ThirdOrderFull=Yes \n HubbardDerivs { \n')
        if containsCarbon == 1:
                outfile.write('  C = -0.1492 \n')
        if containsHydrogen==1:
                outfile.write('  H = -0.1857 \n')
        if containsOxygen==1:
                outfile.write('  O = -0.1575 \n')
        if containsNitrogen==1:
                outfile.write('  N = -0.1535 \n')
        if containsPhosphorus==1:
                outfile.write('  P = -0.14/n')
        if containsSulfur==1:
                outfile.write('  S = -0.11 \n')
        if containsChlorine==1:
                outfile.write('  Cl = -0.0697 \n')
        if containsBromine==1:
                outfile.write('  Br = -0.0573 \n')
        if containsMagnesium==1:
                outfile.write('  Mg = -0.02 \n')
        if containsZinc==1:
                outfile.write('  Zn = -0.03 \n')
        outfile.write('  }')
        outfile.write('\n')
        outfile.write(' HCorrection = Damping {\n     Exponent = 4.00 \n     }')
        outfile.write('\n')
        outfile.write(' Dispersion = DftD3 { \n')
        outfile.write('   Damping = BeckeJohnson { \n')
        outfile.write('     a1 = 0.5719 \n')
        outfile.write('     a2 = 3.6017 \n')
        outfile.write('     } \n')
        outfile.write('  s6 = 1.0  \n')
        outfile.write('  s8 = 0.5883 \n')
        outfile.write('  } \n')
        outfile.write('\n')
        outfile.write(' Solvation = Cosmo {\n')
        outfile.write('    Solvent = FromConstants {\n')
        outfile.write('        Epsilon = 80.2\n')
        outfile.write('        MolecularMass [amu] = 18.0\n')
        outfile.write('        Density [kg/l] = 1.0\n')
        outfile.write('    }\n')
        outfile.write('    FreeEnergyShift [kcal/mol] = 0.0\n')
        outfile.write('    Radii = VanDerWaalsRadiiD3 {}\n')
        outfile.write('    RadiiScaling = 1.55\n')
        outfile.write('    AngularGrid = 110\n')
        outfile.write('    Solver = DomainDecomposition {\n')
        outfile.write('        MaxMoment = 10\n')
        outfile.write('        Accuracy = 1e-8\n')
        outfile.write('        Regularisation = 0.2\n')
        outfile.write('    }\n')
        outfile.write(' }\n')
        outfile.write('\n')
        outfile.write(' Charge = 0 \n')
        outfile.write('} \n')
        outfile.write('Options{} \n')
        outfile.write('Analysis{} \n')
        outfile.write('ParserOptions{} \n')
        outfile.close()
        print('Complete!', 'Generated an input file')

        main()

In [38]:
def read_file_to_list(file_path):
    """Reads lines from a file and returns them as a list.

    Args:
        file_path: The path to the file.

    Returns:
        A list of strings, where each string is a line from the file.
        Returns an empty list if the file does not exist or is empty.
    """
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            # Remove trailing newlines from each line
            lines = [line.rstrip('\n') for line in lines]
            return lines
    except FileNotFoundError:
        print(f"Error: File not found at path: {file_path}")
        return []



In [41]:
protein_list = []
protein_input='/Users/ntw/Desktop/MHT_Calculations/Protein_list.txt'
protein_list=read_file_to_list(protein_input)

for i in range(len(protein_list)):
    pdb_id = protein_list[i]
    pdb_file = pdb_id +".xyz"
    base_directory= "/Users/ntw/Desktop/MHT_Calculations/"
    output_dir = base_directory +pdb_id
    pdb_data = fetch_pdb(pdb_id)
    with open (output_dir+".pdb", "w") as file:
        file.write(pdb_data)
    #Add required hydrogens
make_ChimeraX_Scripts(base_directory)
    
    
    
    #xyz_data = pdb_to_xyz(pdb_data, pdb_id)
    #save_xyz(output_dir, pdb_id, xyz_data)
    #Now, check for duplicated atoms
    #input_filename = os.path.join(output_dir, f"{pdb_id.upper()}.xyz")
    #output_filename = os.path.join(output_dir, f"{pdb_id.upper()}_corrected.xyz")
    #process_xyz(input_filename, output_filename)
    #Add required hydrogens
    
    
    
    #xyz_filename = output_dir + "/" + pdb_id + "_corrected.xyz"
    #generate_swapped_files(xyz_filename, output_dir)
    #remove_br_cl_atoms(xyz_filename, output_dir)
    #move_files_to_subdirectories(output_dir)



#for i in range(len(protein_list)):
#    pdb_id = protein_list[i]  # Example: "1TUP"
#    pdb_file = pdb_id +".xyz"
#    output_dir = "/Users/ntw/Desktop/MHT_Calculations/" +pdb_id
#    pdb_data = fetch_pdb(pdb_id)
#    xyz_data = pdb_to_xyz(output_dir+pdb_file, pdb_id)
#    xyz_data = pdb_to_xyz(output_dir+"/full_pdb_added_h.pdb", pdb_id)
#    #print(xyz_data)
#    print(output_dir+"/full_pdb_added_h.pdb")
#    save_xyz(output_dir, pdb_id, xyz_data)
#    xyz_filename = output_dir + "/" +pdb_file
#    print(xyz_filename)
#    generate_swapped_files(xyz_filename, output_dir)
#    remove_br_cl_atoms(xyz_filename, output_dir)
#    move_files_to_subdirectories(output_dir)
    #with open(output_dir+ "/full_pdb.pdb", 'w') as file:
        #file.write(pdb_data)  # First line: number of atoms
    #make_ChimeraX_Scripts("/Users/ntw/Desktop/MHT_Calculations/")

/Users/ntw/Desktop/MHT_Calculations/2VQT.pdb
/Users/ntw/Desktop/MHT_Calculations/3GW6.pdb
/Users/ntw/Desktop/MHT_Calculations/5LXR.pdb
/Users/ntw/Desktop/MHT_Calculations/6RO3.pdb
/Users/ntw/Desktop/MHT_Calculations/4H3S.pdb
/Users/ntw/Desktop/MHT_Calculations/6KTZ.pdb
/Users/ntw/Desktop/MHT_Calculations/1N11.pdb
/Users/ntw/Desktop/MHT_Calculations/7X09.pdb
/Users/ntw/Desktop/MHT_Calculations/6ZVO.pdb
/Users/ntw/Desktop/MHT_Calculations/2VQU.pdb
/Users/ntw/Desktop/MHT_Calculations/4GOY.pdb
/Users/ntw/Desktop/MHT_Calculations/2VPS.pdb
/Users/ntw/Desktop/MHT_Calculations/6ZWI.pdb
/Users/ntw/Desktop/MHT_Calculations/1K63.pdb
/Users/ntw/Desktop/MHT_Calculations/1K6E.pdb
/Users/ntw/Desktop/MHT_Calculations/4MO1.pdb
/Users/ntw/Desktop/MHT_Calculations/3TUU.pdb
/Users/ntw/Desktop/MHT_Calculations/4OWB.pdb
/Users/ntw/Desktop/MHT_Calculations/2VJX.pdb
/Users/ntw/Desktop/MHT_Calculations/7KQ9.pdb
/Users/ntw/Desktop/MHT_Calculations/6KW0.pdb
/Users/ntw/Desktop/MHT_Calculations/7F9U.pdb
/Users/ntw

Atoms Br at (19.523, 36.504, 2.969) and C at (19.464, 36.512, 3.176) are 0.2154 Ångströms apart


/Users/ntw/Desktop/MHT_Calculations/4GOV/4GOV/4GOV.xyz
Atoms N at (54.561, -23.048, -33.096) and N at (54.561, -23.048, -33.096) are 0.0000 Ångströms apart
Atoms C at (54.129, -24.396, -32.753) and C at (54.137, -24.386, -32.733) are 0.0237 Ångströms apart
Atoms C at (53.398, -25.04, -33.94) and C at (53.376, -25.015, -33.892) are 0.0584 Ångströms apart
Atoms H at (54.632, -22.798, -34.072) and H at (54.627, -22.804, -34.074) are 0.0081 Ångströms apart
Atoms H at (53.444, -24.341, -31.907) and H at (53.471, -24.32, -31.873) are 0.0482 Ångströms apart
Atoms H at (54.111, -25.16, -34.756) and H at (54.051, -25.115, -34.742) are 0.0763 Ångströms apart
Atoms N at (17.644, -12.002, -6.369) and N at (17.644, -12.002, -6.369) are 0.0000 Ångströms apart
Atoms C at (17.821, -11.371, -5.07) and C at (17.843, -11.386, -5.066) are 0.0269 Ångströms apart
Atoms C at (17.303, -9.935, -5.079) and C at (17.406, -9.918, -5.05) are 0.1083 Ångströms apart
Atoms H at (16.71, -12.112, -6.738) and H at (16.7

KeyboardInterrupt: 